In [1]:
"""Import all the relevant libraries."""

from mkd_blockchain import MKDBlockchain
from blockchain_utils import BlockchainUtils
import block
from pubsub import pub
from node import Node
from transaction import Transaction
from transaction_pool import TransactionPool
from wallet import Wallet
from Crypto.PublicKey import RSA
import pprint
import random
import time
from mkdtree import kdtree


In [2]:
"""Initialize Nodes with Genesis Block"""

print('START')
MAX_NODES = 9999
num_nodes = 10
forging_interval = 10  # Time interval.  10 units of time
nodes = [] #list of all nodes
blockchain_dimensions = 4
genesis_node_id = int(num_nodes/2)

#itterate through and initialize each node
then = time.time()
for i in range(num_nodes):
    new_node = Node(i+1, 0)
    #new_node.blockchain = mkd_blockchain
    new_node.start_listener('c0')
    nodes.append(new_node)
now = time.time()-then
print(f'block init time @ {num_nodes} nodes: {now}')


genesis_forger = nodes[genesis_node_id-1].wallet.public_key_string()
mkd_blockchain = MKDBlockchain(blockchain_dimensions, genesis_node_id, genesis_forger)


publisher = nodes[genesis_node_id-1]

start_broadcast = time.time()
publisher.publish(mkd_blockchain)
completed_broadcast = time.time() - start_broadcast
print(f'time to complete broadcast to {num_nodes} nodes: {completed_broadcast}')


START
block init time @ 10 nodes: 35.0290949344635
c1 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
c2 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
c3 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
c4 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
c5 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
c6 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
c7 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
c8 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
c9 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
c10 received payload: <mkd_blockchain.MKDBlockchain object at 0x000001D9E0A27730>
time to complete broadcast to 10 nodes: 0.001997709274291992


In [6]:
"""Attempting to add a second block to the kdtree_blockchain"""

attempt_node_id = 5
attempt_node = nodes[attempt_node_id-1]

attempt_node.test_forge()

kdtree.visualize(attempt_node.blockchain.blocks)
print(f'Root subtreeHash attempt: {nodes[4].blockchain.blocks.subtree_hash}')

attempt_node_id = 1
attempt_node = nodes[attempt_node_id-1]

attempt_node.test_forge()

kdtree.visualize(attempt_node.blockchain.blocks)
print(f'Root subtreeHash attempt: {nodes[4].blockchain.blocks.subtree_hash}')

attempt_node_id = 4
attempt_node = nodes[attempt_node_id-1]

attempt_node.test_forge()

kdtree.visualize(attempt_node.blockchain.blocks)
print(f'Root subtreeHash attempt: {nodes[4].blockchain.blocks.subtree_hash}')


attempt_node_id = 6
attempt_node = nodes[attempt_node_id-1]

attempt_node.test_forge()

kdtree.visualize(attempt_node.blockchain.blocks)
print(f'Root subtreeHash attempt: {nodes[4].blockchain.blocks.subtree_hash}')


I am the forger: 
Traversed kd Nodes: [KDNode(Block([5, 0, 0, 1626126904.509562], [],
0,
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA5RUj6pmwNeH7KtVrtWOo
8OKs1f7lszewtbXFeI2y8uqbNPOM+D2nsWigM84jJ8yKofyxeHIn4Fm18/VWj0PE
u2OZsZ+PFjxHK3JpMgxnC1kaOdKSqbD0hjnC9U9StMkFLLDP0IAK5QphaqDjbxMF
STcA/gnBiMtVDd5F1NxdAL3BqO9yR7As+kMKtlSiFjuQTdX5PSaz2Gx7oj2jpUWp
BOw4jdDXSCoGQpWkUcBuaqRCaRqb9UOEryzCJMPBfanVuzJPbkqEnJC9LL/UPVtY
HCENSe+0qy++IAFby9M53fKfPU0UlCyhzbpteHMTYcmVN8S59f/YkQdaI1Yo2iEN
IQIDAQAB
-----END PUBLIC KEY-----,
), 0, None, 0, KDNode(Block([5, 140, 564, 1626126904.490598], [],
54392be97c88a4a32b249d2fec1fa44021c8cdf4,
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA5RUj6pmwNeH7KtVrtWOo
8OKs1f7lszewtbXFeI2y8uqbNPOM+D2nsWigM84jJ8yKofyxeHIn4Fm18/VWj0PE
u2OZsZ+PFjxHK3JpMgxnC1kaOdKSqbD0hjnC9U9StMkFLLDP0IAK5QphaqDjbxMF
STcA/gnBiMtVDd5F1NxdAL3BqO9yR7As+kMKtlSiFjuQTdX5PSaz2Gx7oj2jpUWp
BOw4jdDXSCoGQpWkUcBuaqRCaRqb9UOEryzCJMPBfanVuzJPbkqEnJC9LL/UPVtY

In [8]:
test_block = nodes[5].blockchain.blocks.left.right.data
print(test_block)
print(f'Parent Hash: {BlockchainUtils.hash(test_block.to_json()).hexdigest()}')
child_block = nodes[5].blockchain.blocks.left.right.right.data
print(child_block)

Block([4, 427, 744, 1626126904.490598], [],
d3cb76654a16d4e2c427c75997b283adb429bb8c,
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA3PFEQsOQABGMAQDnXNMG
rzMS8OXFVNxsX+l0jbSUhIWMe11AP7AQroDGs/LJMyjcDSpO+QPcT6UrEPzI6ZZH
ygwPCEtxnvb00K9Mm7kSvwmw4gO4HtKhAoCY15Xc1kkx4nE0f5U8ObPcTj/gTe6F
/L9b/Nz/Vj1dCOPMzHLoHS+K/bprWPdIy/3fdK9RYrbKihGOoMIaSXvTU69jpjXm
O9//i/e33nE4eu3LTAe+IdHJRf9nDkWltu8nmruFB8s7JvpnW8/m80NoSE+tRmL2
4/XaYOOhHsdjLYvcUn0wA5gr5nf0UmbMbVo1KmnOrZnqbe0qCK+KxMKSSz0YS5o9
2QIDAQAB
-----END PUBLIC KEY-----,
5cc1f029e2a0be142c77cabc4cc35c7a9b52536bc3ea9eec8d688529586aafb2e8b2479bf42190057b70713d888e9eea827ab368f27f14a38027ada4f1b725aef88161231462afbdda8d0467a9d0f85f4dd3439d55f4eb7ca89871fe1770d3f0ecfc088f30c4199b822b8d97d9a18c54b2a331e1626ad9ea5271915a09442c352720c8144bc5d10bf57c63b3747050b55726999f44b72e0dccce4be7ac52753798a293039537372204c5c4664a544987195052587ec4a488cae73a920073d3c7fd89ca6fc9e939fdf141efe8c2659d64d84b8b896dcb3c7735f5ade86c4c2f8a334a7c275cbe00

In [ ]:
# TODO: Need to generate random transactions and need to start implementing test data.


In [5]:
#Opens the data set file and generates nodes based on the provided data.
#file is comma delimited in the following format: time-nodeID-xcoor-ycoor-miner-region-prevregion
then = time.time()
with open('node_data.txt') as f:
    lines = f.readlines() # list containing lines of file
    i = 0

    for line in lines:
        j = i%80
        print(j)
        parts = line.split(',')
        moving_node = nodes[j]

        old_topic = 'c'+str(moving_node.cluster_id).strip()
        new_topic = 'c' + str(parts[5]).strip()

        moving_node.coords = [parts[2], parts[3]]

        moving_node.cluster_id = parts[5]
        moving_node.move_listener(old_topic, new_topic)

        #merge right after movement

        #variable that sends transaction if miner==1
        if parts[4] == 1:
            # initiate transaction, save it to pool and broadcast.
            x=1

        # if time to forge then forge and broadcast, needs to scan and perform all clusters
            # if transaction pool not empty then forge and broadcast


        # move_listener(node[i % num_nodes].node_listener, "c" + str(parts[6]).strip(), "c" + str(parts[5]).strip())

        i += 1
now = time.time() - then
print(f'node movement time @ {num_nodes} nodes: {now}')

0
1
2
3
4
5
6
7
8
9
10


IndexError: list index out of range